In [2]:
import torch
import numpy as np
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import csr_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

In [3]:
df_dev = pd.read_json('./Twibot-20/dev.json')

# print(df_dev.head())
print('loaded df_dev')

loaded df_dev


In [4]:
print("number of users in dev", df_dev.shape[0])
print("num tweets in dev", df_dev['tweet'].apply(lambda x: len(x) if isinstance(x, list) else 0).sum())

number of users in dev 2365
num tweets in dev 401525


In [5]:
# tweets=[]
# for i in range (df_dev.shape[0]):
#     one_usr_tweets=[]
#     if df_dev['tweet'][i] is None:
#         one_usr_tweets.append('')
#     else:
#         for each in df_dev['tweet'][i]:
#             one_usr_tweets.append(each)
#     tweets.append(one_usr_tweets)


# tweets = df_dev['tweet'].apply(lambda x: np.array(x) if isinstance(x, list) else np.array([]))

# tweets = pd.DataFrame(df_dev[df_dev.tweet.notnull()]['tweet'].values.tolist()).values
# print(type(tweets))
from tqdm import tqdm 


tweets = df_dev[df_dev.tweet.notnull()]['tweet'].values
print(type(tweets))
print(tweets.shape)
print(type(tweets[0]))
print("testing")

print("dimensionality of tweets np array", tweets.shape)

print('loading tf-idf + truncated SVD')
print('extracting tf-idf matrix')

vectorizer = TfidfVectorizer()

# vectorizer.fit(tweets.flatten())
print('fitting tf-idf matrix')
vectorizer.fit(np.concatenate(tweets))

print('transforming tf-idf matrix')
tf_idf_matrix = np.array([vectorizer.transform(usrtweets) for usrtweets in tqdm(tweets)])
print('transformed tf-idf matrix')
print('dimensionality of tf-idf matrix', tf_idf_matrix.shape)
print('type of elements in tf-df matrix', type(tf_idf_matrix[0]))



<class 'numpy.ndarray'>
(2350,)
<class 'list'>
testing
dimensionality of tweets np array (2350,)
loading tf-idf + truncated SVD
extracting tf-idf matrix
fitting tf-idf matrix


KeyboardInterrupt: 

In [ ]:
print('dimensionality of first element in tf-idf matrix', tf_idf_matrix[0].shape)
print('dimensionality of second element in tf-idf matrix', tf_idf_matrix[1].shape)

dimensionality of first element in tf-idf matrix (99, 608913)
dimensionality of second element in tf-idf matrix (200, 608913)


In [ ]:
from scipy.sparse import vstack


# csr_tfidf_matrix = csr_matrix(tf_idf_matrix)

# print("tf-idf matrix dimensions:", tf_idf_matrix.shape)
# print('fitting truncated SVD')
svd = TruncatedSVD(n_components=100, n_iter=5, random_state=42) # n_iter=5 is the default, n_components=2 is the default but we want to use 100 components (to test it out)

print('fitting truncated SVD')
svd.fit(vstack(tf_idf_matrix))

# trunc_svd_matrix = svd.fit_transform(csr_tfidf_matrix)
# print('dimensionality of tweets truncated svd matrix:',trunc_svd_matrix.shape)
# print(tweets)

fitting truncated SVD


TruncatedSVD(n_components=100, random_state=42)

In [ ]:
user_tweet_svds = np.array([svd.transform(usertweettfidf) for usertweettfidf in tf_idf_matrix])

/Users/kareem/.local/lib/python3.7/site-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  """Entry point for launching an IPython kernel.


In [ ]:
averaged_tweet_embeddings = np.array([np.mean(usertweetembeddings, axis=0) for usertweetembeddings in user_tweet_svds])
print('dimensionality of averaged tweet embeddings', averaged_tweet_embeddings.shape)

dimensionality of averaged tweet embeddings (2350, 100)


In [ ]:
# extracting follower counts into separate tensor

numerical_feature_names = ['followers_count', 'friends_count','favourites_count','statuses_count']

for feature in numerical_feature_names:
    torch.save(torch.tensor([0 if (df_dev['profile'][i] is None or df_dev['profile'][i][feature] is None) else float(df_dev['profile'][i][feature]) for i in range(df_dev.shape[0])], dtype=torch.float32),'./Data/DatasetPlaygroundResources/'+feature+'.pt')

followers_count, friends_count, favourites_count, statuses_count = [torch.load('./Data/DatasetPlaygroundResources/'+feature+'.pt') for feature in numerical_feature_names]


